In [26]:
import pandas as pd
import numpy as np

In [27]:
#Linear Regression Code From The Lesson wuth a regularization option
def train_linear_regression(X, y,r):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0])
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

In [28]:
#function taken from the lecture to compute the MSE error
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [29]:
#Reading the Dataset
data_frame=pd.read_csv("housing.csv")
data_frame.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [30]:
#keeping only the relevant columns
df=data_frame[['latitude','longitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income','median_house_value']]

In [31]:
#Checking null values
df.isnull().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

In [32]:
#Calculating the median value for population
print("The median for population is "  , df['population'].median())

The median for population is  1166.0


In [33]:
#Shuffling the data and fixing the seed
index=np.arange(len(df))
np.random.seed(42)
np.random.shuffle(index)

In [34]:
#Spliting the data into training, validation and testing
n_val = int(len(df) * 0.2)
n_test = int(len(df) * 0.2)
n_train = len(df) - n_val - n_test
train_data=df.iloc[index[:n_train]]
val_data=df.iloc[index[n_train:n_train+n_val]]
test_data=df.iloc[index[n_train+n_val:]]


In [35]:
#Apply log transformation to the Median House Value and assigining it as Y then dropping it from the datafarmes
y_train = np.log1p(train_data['median_house_value'])
y_val = np.log1p(val_data['median_house_value'])
y_test = np.log1p(test_data['median_house_value'])
del train_data['median_house_value']
del val_data['median_house_value']
del test_data['median_house_value']

In [36]:
#Let's try the first filling Nan with zeros
train_data =train_data.fillna(0)
#Apply Linear Regression
w0, w = train_linear_regression(train_data, y_train,0)

In [37]:
#Let's calculate the RMSE for validation data, fill first the Nan values with zeros and then fitting for prediction,then 
#calculating the rounded validation error
val_data =val_data.fillna(0)
Ypred_val= w0 + val_data.dot(w)
valid_error_zero=round(rmse(y_val,Ypred_val),2)



In [38]:
#Let's now repeat the procedure again to replace instead of zeros average values of the training data
train_data=df.iloc[index[:n_train]]
val_data=df.iloc[index[n_val:n_train+n_val]]
test_data=df.iloc[index[n_train+n_val:]]
y_train = np.log1p(train_data['median_house_value'])
y_val = np.log1p(val_data['median_house_value'])
y_test = np.log1p(test_data['median_house_value'])
del train_data['median_house_value']
del val_data['median_house_value']
del test_data['median_house_value']


In [39]:
#calculate the mean of the training value
##Now let's do the linear regression part and calculate the validation error and compare with the first approach

mean_train=train_data['total_bedrooms'].mean()
train_data =train_data.fillna(mean_train)
w0, w = train_linear_regression(train_data, y_train,0)
val_data =val_data.fillna(mean_train)
Ypred_val= w0 + val_data.dot(w)
valid_error_average=round(rmse(y_val,Ypred_val),2)



In [40]:
print("Validation error with replacing Nan with zeros is", valid_error_zero)
print("Validation error with replacing Nan with average is", valid_error_average)


Validation error with replacing Nan with zeros is 0.33
Validation error with replacing Nan with average is 0.33


In [41]:
#As it can be seen,  the values of the errors are equal to the second decimal digit

In [42]:
#Question 4
#let us now try playing with the tuning parameter r
R=np.array([0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10])
#This is for holding the error
RMSEr=np.zeros((len(R)))

#Again let us split the data into training validation and testing 
train_data=df.iloc[index[:n_train]]
val_data=df.iloc[index[n_train:n_train+n_val]]
test_data=df.iloc[index[n_train+n_val:]]
y_train = np.log1p(train_data['median_house_value'])
y_val = np.log1p(val_data['median_house_value'])
y_test = np.log1p(test_data['median_house_value'])
del train_data['median_house_value']
del val_data['median_house_value']
del test_data['median_house_value']
train_data =train_data.fillna(0)
val_data =val_data.fillna(0)


In [43]:

#looping over every r and calculating the RMSE Error to a second decimal digit
for i in range(len(R)):
    w0, w = train_linear_regression(train_data, y_train,R[i])
    Ypred_val= w0 + val_data.dot(w)
    RMSEr[i]=round(rmse(y_val,Ypred_val),2)


In [70]:
RMSEr

array([0.33, 0.33, 0.33, 0.33, 0.33, 0.33, 0.33, 0.34, 0.34])

In [44]:
#calculating the values with minimum error
ind=np.where(RMSEr == np.amin(RMSEr))
print("The r with minimum RMSE is", R[ind])

The r with minimum RMSE is [0.e+00 1.e-06 1.e-04 1.e-03 1.e-02 1.e-01 1.e+00]


In [45]:
#The smallest r is zero

In [46]:
#Varying the Seed
seed=np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
RMSEs=np.zeros((len(seed)))

In [47]:
for i in range(len(seed)):
    index=np.arange(len(df))
    #change the seed
    np.random.seed(seed[i])
    np.random.shuffle(index)
    #splitting into training and valuidation
    train_data=df.iloc[index[:n_train]]
    val_data=df.iloc[index[n_train:n_train+n_val]]
    test_data=df.iloc[index[n_train+n_val:]]
    y_train = np.log1p(train_data['median_house_value'])
    y_val = np.log1p(val_data['median_house_value'])
    y_test = np.log1p(test_data['median_house_value'])
    del train_data['median_house_value']
    del val_data['median_house_value']
    del test_data['median_house_value']
    train_data=train_data.fillna(0)
    val_data=val_data.fillna(0)
    test_data=test_data.fillna(0)
    #apply linear regression
    w0, w = train_linear_regression(train_data, y_train,0)
    Ypred_val= w0 + val_data.dot(w)
    #calculate RMSE
    RMSEs[i]=round(rmse(y_val,Ypred_val),2)
    

    
    

In [48]:
#calculating standard deviation
Standard_deviation=round(np.std(RMSEs),3)

In [49]:
Standard_deviation

0.005

In [50]:
#Question 6
np.random.seed(9)
#spliting the training and testing
np.random.shuffle(index)
train_data=df.iloc[index[:n_train]]
val_data=df.iloc[index[n_train:n_train+n_val]]
test_data=df.iloc[index[n_train+n_val:]]
y_train = np.log1p(train_data['median_house_value'])
y_val = np.log1p(val_data['median_house_value'])
y_test = np.log1p(test_data['median_house_value'])
del train_data['median_house_value']
del val_data['median_house_value']
del test_data['median_house_value']

In [51]:
#Merge Validation and Training Data sets
Total_train= train_data.append(val_data)
Total_train=Total_train.fillna(0)
Y_train_total=np.concatenate((y_train,y_val),axis=0)


In [52]:
#learning the coefficinets and calculate the testing error
w0, w = train_linear_regression(Total_train, Y_train_total,0.001)
test_data=test_data.fillna(0)
Y_test= w0 + test_data.dot(w)
Testing_error=round(rmse(y_test,Y_test),2)

In [53]:
print(Testing_error)

0.35
